----------------------------------------------------------------------------------------------------------------------

In [1]:
%pip install -r requirements.txt


Note: you may need to restart the kernel to use updated packages.


In [2]:
"""
Gelişmiş öğrenme koçu.
Bu betik, öğrencinin dikkat ve etkileşim verilerini gerçek zamanlı olarak toplayarak
ChromaDB'de etiketli bir şekilde saklar ve heuristik kurallara göre geri bildirim üretir.
"""

from dotenv import load_dotenv
import os
import requests
import time
import pyttsx3
import random
import threading

from reportlab.pdfgen import canvas
from reportlab.lib.pagesizes import A4
from langchain_google_genai import GoogleGenerativeAI, GoogleGenerativeAIEmbeddings
from langchain_community.vectorstores import Chroma
from langchain.chains import RetrievalQA
from langchain.agents import Tool, initialize_agent
from langchain.agents.agent_types import AgentType
from flask import Flask, request, jsonify, send_file
from flask_cors import CORS
from datetime import datetime


c:\Users\Hp\.conda\envs\AI_Agent\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


----------------------------------------------------------------------------------------------------------------------

In [3]:
load_dotenv()
STUDENT_ID = "ogrenci_001"

# Google API key kontrolü
api_key = os.getenv("GOOGLE_API_KEY")
if not api_key:
    print("❌ GOOGLE_API_KEY bulunamadı! .env dosyasını kontrol edin.")
    raise ValueError("Google API anahtarı gerekli")

print("✅ Google API anahtarı yüklendi")

try:
    LLM = GoogleGenerativeAI(model="gemini-2.0-flash-exp", temperature=0.1)
    embedding = GoogleGenerativeAIEmbeddings(model="models/embedding-001")
    print("✅ Google AI modelleri başarıyla yüklendi")
except Exception as e:
    print(f"❌ Google AI model yükleme hatası: {e}")
    print("🔄 Alternatif model deneniyor...")
    try:
        LLM = GoogleGenerativeAI(model="gemini-1.5-flash", temperature=0.1)
        print("✅ Alternatif model yüklendi")
    except Exception as e2:
        print(f"❌ Alternatif model de başarısız: {e2}")
        raise

try:
    vectordb = Chroma(persist_directory="student_memory", embedding_function=embedding)
    print("✅ ChromaDB başarıyla başlatıldı")
except Exception as e:
    print(f"❌ ChromaDB başlatma hatası: {e}")
    # Yeni bir ChromaDB oluşturmayı dene
    import shutil
    try:
        shutil.rmtree("student_memory", ignore_errors=True)
        vectordb = Chroma(persist_directory="student_memory", embedding_function=embedding)
        print("✅ ChromaDB yeniden oluşturuldu")
    except Exception as e2:
        print(f"❌ ChromaDB yeniden oluşturma hatası: {e2}")
        raise

try:
    qa_chain = RetrievalQA.from_chain_type(
        llm=LLM,
        retriever=vectordb.as_retriever(search_kwargs={"k": 3}),
        chain_type="stuff"
    )
    print("✅ QA Chain başarıyla oluşturuldu")
except Exception as e:
    print(f"❌ QA Chain oluşturma hatası: {e}")
    raise

✅ Google API anahtarı yüklendi
✅ Google AI modelleri başarıyla yüklendi


C:\Users\Hp\AppData\Local\Temp\ipykernel_29520\963484775.py:27: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-chroma package and should be used instead. To use it run `pip install -U :class:`~langchain-chroma` and import as `from :class:`~langchain_chroma import Chroma``.
  vectordb = Chroma(persist_directory="student_memory", embedding_function=embedding)


✅ ChromaDB başarıyla başlatıldı
✅ QA Chain başarıyla oluşturuldu


----------------------------------------------------------------------------------------------------------------------

In [4]:
# Veri akışını sağlayan endpointler
ENDPOINT_ATTENTION = "http://127.0.0.1:8001/attention"
ENDPOINT_SCRIPT = "http://localhost:5002/get_texts"
ENDPOINT_KYBRD_MOUSE_INTERRUPT = "http://localhost:5001/api/status"


----------------------------------------------------------------------------------------------------------------------

In [5]:

def dikkat_uyarisi_ver(_):
    try:
        recent = vectordb.similarity_search("attention", k=1)[0]
        data = recent.metadata
        if not data.get("screen", False) or data.get("attention", 0) < 0.5:
            print("⚠️ Dikkatin dağıldı! Lütfen odaklan.")
            try:
                import winsound
                winsound.Beep(1000, 400)
            except:
                pass
            print("\033[91m[Çerçeve RENGİ: KIRMIZI - ODAK YOK]\033[0m")
    except Exception as e:
        print(f"🚨 Dikkat modülü hatası: {e}")

def mola_onerisi_chromadb(_):
    try:
        recent_docs = vectordb.similarity_search("attention", k=10)
        att_20s = [
            doc.metadata.get("att_20min", 0)
            for doc in recent_docs
            if isinstance(doc.metadata.get("att_20min"), (float, int))
        ]
        if len(att_20s) >= 3 and sum(att_20s) / len(att_20s) < 0.6:
            print("😴 20dk ortalaman düşük. Mola vermelisin.")
            try:
                import winsound
                winsound.Beep(800, 500)
            except:
                pass
            print("\033[95m[Mola Önerisi: Düşük dikkat algılandı.]\033[0m")
    except Exception as e:
        print(f"🚨 Mola modülü hatası: {e}")

def periyot_onerisi_llm_ileri(_):
    try:
        recent_docs = vectordb.similarity_search("attention", k=15)
        att_vals = [doc.metadata.get("attention", 0) for doc in recent_docs]
        focus_vals = [doc.metadata.get("focus_score", 0) for doc in recent_docs]
        attention_summary = "\n".join([
            f"- Dikkat: {round(doc.metadata.get('attention', 0),2)} "
            f"| 20dk: {round(doc.metadata.get('att_20min', 0),2)} "
            f"| Focus: {round(doc.metadata.get('focus_score', 0),2)}"
            for doc in recent_docs
        ])
        if len(att_vals) < 5:
            print("🔁 Kişisel öneri için yeterli veri yok.")
            return

        prompt = f"""
🧠 Öğrenci dikkat geçmişi:
{attention_summary}

📊 Ortalama dikkat: {round(sum(att_vals)/len(att_vals), 2)}
🎙️ Ortalama focus: {round(sum(focus_vals)/len(focus_vals), 2)}

Bu öğrenciye uygun öğrenme periyodu ve stratejileri öner.
"""
        feedback = qa_chain.run(prompt)
        print("\n🧠 Öğrenme Önerisi:\n", feedback)
    except Exception as e:
        print(f"🚨 Öğrenme önerisi hatası: {e}")

def zihin_yorgunlugu_tahmini(_):
    try:
        recent_docs = vectordb.similarity_search("attention", k=10)
        if len(recent_docs) < 5:
            print("🔁 Yorgunluk analizi için yeterli veri yok.")
            return

        attention_values = [doc.metadata.get("attention", 0) for doc in recent_docs]
        first_half_avg = sum(attention_values[:3]) / 3
        last_half_avg = sum(attention_values[-3:]) / 3
        delta = first_half_avg - last_half_avg

        if delta > 0.15:
            print("🧠 Zihin yorgunluğu tespit edildi. Dikkatin ciddi oranda düştü.")
            try:
                import winsound
                winsound.Beep(600, 500)
            except:
                pass
            print("\033[93m[Yorgunluk Algılandı: Kısa bir ara vermelisin.]\033[0m")
    except Exception as e:
        print(f"🚨 Yorgunluk tahmin modülü hatası: {e}")

def vak_ogrenme_tarzi_tahmini(_):
    try:
        recent_docs = vectordb.similarity_search("attention", k=10)
        visual = auditory = kinesthetic = 0

        for doc in recent_docs:
            m = doc.metadata
            if m.get("screen"): visual += 1
            if m.get("focus_score", 0) > 0.6: auditory += 1
            if m.get("keyboard_activity") or m.get("mouse_activity"): kinesthetic += 1

        toplam = visual + auditory + kinesthetic
        if toplam == 0:
            print("🔁 Öğrenme tarzı için yeterli veri yok.")
            return

        style = max(
            (visual, "Görsel"),
            (auditory, "İşitsel"),
            (kinesthetic, "Kinestetik"),
        )[1]
        print(f"📚 Tahmini öğrenme tarzın: \033[94m{style}\033[0m")
    except Exception as e:
        print(f"🚨 Öğrenme tarzı modülü hatası: {e}")

def oturum_ozet_raporu(_):
    try:
        recent_docs = vectordb.similarity_search("attention", k=10)
        if not recent_docs:
            print("🔁 Özet için yeterli veri yok.")
            return

        avg_att = sum(doc.metadata.get("attention", 0) for doc in recent_docs) / len(recent_docs)
        avg_focus = sum(doc.metadata.get("focus_score", 0) for doc in recent_docs) / len(recent_docs)
        en_focused = max(recent_docs, key=lambda d: d.metadata.get("focus_score", 0))
        saat = en_focused.metadata.get("timestamp", "bilinmiyor")

        print("\n📊 \033[92mOturum Özeti:\033[0m")
        print(f"- Ortalama dikkat: {round(avg_att, 2)}")
        print(f"- Ortalama focus: {round(avg_focus, 2)}")
        print(f"- En odaklı zaman: {saat}")
    except Exception as e:
        print(f"🚨 Oturum özeti hatası: {e}")

def sesli_motivasyon_ver(mesaj="Harika gidiyorsun! Odaklanmaya devam et."):
    try:
        engine = pyttsx3.init()
        engine.setProperty('rate', 175)
        engine.say(mesaj)
        engine.runAndWait()
    except Exception as e:
        print(f"[Sesli Uyarı Hatası] {e}")

def analiz_sorudan_anlam_cikar_ve_yanitla(soru: str):
    try:
        recent_docs = vectordb.similarity_search("attention", k=10)
        attention_lines = "\n".join([
            f"- Dikkat: {round(doc.metadata.get('attention', 0), 2)} "
            f"| Focus: {round(doc.metadata.get('focus_score', 0), 2)}"
            for doc in recent_docs
        ])
        prompt = f"""
Kullanıcının sorusu: "{soru}"

Aşağıda kullanıcının son dikkat ve etkileşim verileri var:
{attention_lines}

Bu verilerle birlikte, kullanıcının sorusuna cevap ver:
- Soruyu anlamlandır.
- Eğer performans sorusuysa analiz yap.
- Yorgunluksa belirt.
- Motivasyon istiyorsa cesaretlendir.
- Öğrenme önerisi istiyorsa tavsiye ver.
- Konuşma tonun sıcak ve koçvari olsun.
"""
        yanit = LLM.invoke(prompt)
        return yanit
    except Exception as e:
        return f"[Analiz Hatası] {e}"
    
    
def ozet_ses_logs(_):
    # 1. ham kayıtları oku
    log_path = "system_session_full.txt"
    try:
        with open(log_path, "r", encoding="utf-8") as f:
            text = f.read()
    except Exception as e:
        return f"❌ Log dosyası okunamadı: {e}"

    # 2. LLM ile uzun metni bölüp özet çıkar
    #    Burada qa_chain veya LLM.invoke kullanabilirsiniz
    chunks = [ text[i:i+2000] for i in range(0, len(text), 2000) ]
    summaries = []
    for chunk in chunks:
        prompt = f"""Aşağıdaki zaman damgalı ses kaydını özetle ve önemli noktaları maddele:\n\n{chunk}"""
        summaries.append( qa_chain.run(prompt) )
    full_summary = "\n\n".join(summaries)

    # 3. PDF oluştur
    pdf_path = "session_summary.pdf"
    try:
        c = canvas.Canvas(pdf_path, pagesize=A4)
        width, height = A4
        text_obj = c.beginText(40, height - 40)
        text_obj.setFont("Helvetica", 11)
        for line in full_summary.split("\n"):
            text_obj.textLine(line)
            # sayfa dolarsa yeni sayfa aç
            if text_obj.getY() < 40:
                c.drawText(text_obj)
                c.showPage()
                text_obj = c.beginText(40, height - 40)
                text_obj.setFont("Helvetica", 11)
        c.drawText(text_obj)
        c.save()
    except Exception as e:
        return f"❌ PDF oluşturulurken hata: {e}"

    return f"✅ Özet PDF hazır: {pdf_path}"


----------------------------------------------------------------------------------------------------------------------

In [6]:
# Araçları tanımla
tools = [
    Tool(name="DikkatUyarisi", func=dikkat_uyarisi_ver, description="Dikkat seviyesi düşükse uyarı verir."),
    Tool(name="MolaOnerisi", func=mola_onerisi_chromadb, description="20dk ortalama dikkat düşüklüğünde mola önerir."),
    Tool(name="OgrenmePeriyoduOnerisi", func=periyot_onerisi_llm_ileri, description="Dikkat geçmişine göre öğrenme stratejisi önerir."),
    Tool(name="ZihinYorgunluguTahmini", func=zihin_yorgunlugu_tahmini, description="Son dikkat verilerindeki düşüşe göre zihin yorgunluğu tespit eder."),
    Tool(name="OgrenmeTarziTahmini", func=vak_ogrenme_tarzi_tahmini, description="Kullanıcının VAK öğrenme tarzını tahmin eder."),
    Tool(name="OturumOzeti", func=oturum_ozet_raporu, description="Son verilerle oturum özeti sunar."),
    Tool(name="SoruyaGoreAnalizYap", func=analiz_sorudan_anlam_cikar_ve_yanitla, description="Soruya göre analiz yapar ve LLM ile cevap üretir."),
    Tool(name="SesOzetPDF", func=ozet_ses_logs, description="`system_session_full.txt` dosyasındaki tüm ses kayıtlarını LLM ile özetler ve PDF olarak kaydeder.")
]

# Agent'ı başlat
system_prompt = """
Sen bir eğitim koçusun.
- Sayısal metrik kullanma, sadece nitel ifadeler (çok yüksek/orta/düşük) kullan.
- Konuşman sıcakkanlı, destekleyici ve motive edici olsun.
- Gerekirse araçları (Tools) çağır.
"""
agent = initialize_agent(
    tools=tools,
    llm=LLM,
    agent_type=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True,
    system_message=system_prompt
)

C:\Users\Hp\AppData\Local\Temp\ipykernel_29520\3422287574.py:20: LangChainDeprecationWarning: LangChain agents will continue to be supported, but it is recommended for new use cases to be built with LangGraph. LangGraph offers a more flexible and full-featured framework for building agents, including support for tool-calling, persistence of state, and human-in-the-loop workflows. For details, refer to the `LangGraph documentation <https://langchain-ai.github.io/langgraph/>`_ as well as guides for `Migrating from AgentExecutor <https://python.langchain.com/docs/how_to/migrate_agent/>`_ and LangGraph's `Pre-built ReAct agent <https://langchain-ai.github.io/langgraph/how-tos/create-react-agent/>`_.
  agent = initialize_agent(


----------------------------------------------------------------------------------------------------------------------

In [7]:
# ——————————————
# 2) Yardımcı Fonksiyonlar
# ——————————————
def get_safe_json(url):
    try:
        resp = requests.get(url, timeout=5)
        if resp.status_code == 200 and resp.text.strip():
            return resp.json()
    except:
        pass
    return {}

def kvalitatif_deger(score):
    if score is None: return "ölçülemediği"
    if score >= 0.8:     return "çok yüksek"
    if score >= 0.5:     return "orta"
    return "düşük"



----------------------------------------------------------------------------------------------------------------------

In [ ]:
# Flask API uygulaması
app = Flask(__name__)
CORS(app)

last_agent_response = "Henüz agent çalışmadı."
last_alert = None

@app.route("/download_session_summary", methods=["GET"])
def download_session_summary():
    pdf_path = "session_summary.pdf"
    if not os.path.exists(pdf_path):
        return jsonify({
            "error": "Özet PDF henüz oluşturulmadı. Agent'a SesOzetPDF aracını çalıştır deyin."
        }), 404
    return send_file(pdf_path, as_attachment=True)

@app.route("/last_response", methods=["GET"])
def get_last_response():
    response = {
        "answer": last_agent_response
    }
    if last_alert is not None:
        response["alert"] = last_alert
    return jsonify(response)

@app.route("/ask", methods=["POST"])
def ask_agent():
    global last_agent_response, last_alert

    try:
        data = request.get_json() or {}
        soru = data.get("question", "").strip()
        if not soru:
            return jsonify({"error": "Soru boş olamaz."}), 400

        session_id = datetime.now().strftime("%Y%m%d_%H")
        now_ts = datetime.now().isoformat()

        # — Kullanıcı mesajını kaydet
        try:
            vectordb.add_texts(
                [soru],
                metadatas=[{
                    "user_id": STUDENT_ID,
                    "session_id": session_id,
                    "timestamp": now_ts,
                    "type": "conversation",
                    "speaker": "User"
                }]
            )
        except Exception as e:
            print(f"Veritabanı kayıt hatası: {e}")

        # — Önceki 5 sohbeti getir
        try:
            conv_docs = vectordb.similarity_search(
                soru, k=5,
                filter={"user_id": STUDENT_ID, "type": "conversation"}
            )
            history = "\n".join(f"{d.metadata['speaker']}: {d.page_content}" for d in conv_docs)
        except Exception as e:
            print(f"Geçmiş sohbet alma hatası: {e}")
            history = ""

        # — Context + soru ile prompt oluştur
        prompt = f"""
Önceki konuşmalar:
{history}

Kullanıcı: {soru}
"""

        # Agent'ı güvenli şekilde çalıştır
        try:
            yanit = agent.run(prompt)
        except Exception as agent_error:
            print(f"Agent çalıştırma hatası: {agent_error}")
            # Basit bir yanıt üret
            try:
                yanit = LLM.invoke(f"Eğitim koçu olarak şu soruya cevap ver: {soru}")
            except Exception as llm_error:
                print(f"LLM hatası: {llm_error}")
                yanit = "Üzgünüm, şu anda teknik bir sorun yaşıyorum. Lütfen daha sonra tekrar deneyin."

        # — Agent cevabını kaydet
        try:
            vectordb.add_texts(
                [yanit],
                metadatas=[{
                    "user_id": STUDENT_ID,
                    "session_id": session_id,
                    "timestamp": datetime.now().isoformat(),
                    "type": "conversation",
                    "speaker": "Agent"
                }]
            )
        except Exception as e:
            print(f"Cevap kayıt hatası: {e}")

        # Buraya istersen ask_agent içinde de last_alert reset veya güncelleme ekleyebilirsin
        last_agent_response = yanit

        return jsonify({"answer": yanit})

    except Exception as e:
        print(f"Genel API hatası: {e}")
        error_response = "Teknik bir sorun oluştu. Lütfen daha sonra tekrar deneyin."
        return jsonify({"answer": error_response, "error": str(e)}), 500

# Flask sunucusunu ayrı bir thread'de başlat
def start_flask_server():
    try:
        app.run(host="0.0.0.0", port=8005, debug=False, threaded=True)
    except Exception as e:
        print(f"Flask sunucu başlatma hatası: {e}")

flask_thread = threading.Thread(target=start_flask_server, daemon=True)
flask_thread.start()

print("🚀 Flask API sunucusu başlatıldı: http://localhost:8005")


🚀 Flask API sunucusu başlatıldı: http://localhost:8005


 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:8005
 * Running on http://192.168.1.126:8005
Press CTRL+C to quit
127.0.0.1 - - [06/Aug/2025 15:31:34] "OPTIONS /last_response HTTP/1.1" 200 -
127.0.0.1 - - [06/Aug/2025 15:31:34] "GET /last_response HTTP/1.1" 200 -
127.0.0.1 - - [06/Aug/2025 15:31:35] "GET /last_response HTTP/1.1" 200 -
127.0.0.1 - - [06/Aug/2025 15:31:36] "GET /last_response HTTP/1.1" 200 -
127.0.0.1 - - [06/Aug/2025 15:31:37] "GET /last_response HTTP/1.1" 200 -
127.0.0.1 - - [06/Aug/2025 15:31:37] "OPTIONS /ask HTTP/1.1" 200 -
127.0.0.1 - - [06/Aug/2025 15:31:38] "GET /last_response HTTP/1.1" 200 -


Geçmiş sohbet alma hatası: Expected where to have exactly one operator, got {'user_id': 'ogrenci_001', 'type': 'conversation'} in query.


> Entering new AgentExecutor chain...


C:\Users\Hp\AppData\Local\Temp\ipykernel_29520\1879942637.py:75: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  yanit = agent.run(prompt)
Retrying langchain_google_genai.llms._completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised ResourceExhausted: 429 You exceeded your current quota, please check your plan and billing details. For more information on this error, head to: https://ai.google.dev/gemini-api/docs/rate-limits. [violations {
  quota_metric: "generativelanguage.googleapis.com/generate_content_free_tier_requests"
  quota_id: "GenerateRequestsPerDayPerProjectPerModel-FreeTier"
  quota_dimensions {
    key: "model"
    value: "gemini-2.0-flash-exp"
  }
  quota_dimensions {
    key: "location"
    value: "global"
  }
  quota_value: 50
}
, links {
  description: "Learn more about Gemini API quotas"
  url: "https://ai.google.dev/gemini-api/docs/rate-limits"


Agent çalıştırma hatası: 429 You exceeded your current quota, please check your plan and billing details. For more information on this error, head to: https://ai.google.dev/gemini-api/docs/rate-limits. [violations {
  quota_metric: "generativelanguage.googleapis.com/generate_content_free_tier_requests"
  quota_id: "GenerateRequestsPerDayPerProjectPerModel-FreeTier"
  quota_dimensions {
    key: "model"
    value: "gemini-2.0-flash-exp"
  }
  quota_dimensions {
    key: "location"
    value: "global"
  }
  quota_value: 50
}
, links {
  description: "Learn more about Gemini API quotas"
  url: "https://ai.google.dev/gemini-api/docs/rate-limits"
}
, retry_delay {
  seconds: 49
}
]


127.0.0.1 - - [06/Aug/2025 15:32:10] "OPTIONS /last_response HTTP/1.1" 200 -
127.0.0.1 - - [06/Aug/2025 15:32:10] "GET /last_response HTTP/1.1" 200 -
127.0.0.1 - - [06/Aug/2025 15:32:11] "GET /last_response HTTP/1.1" 200 -
127.0.0.1 - - [06/Aug/2025 15:32:12] "GET /last_response HTTP/1.1" 200 -
127.0.0.1 - - [06/Aug/2025 15:32:13] "GET /last_response HTTP/1.1" 200 -
Retrying langchain_google_genai.llms._completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised ResourceExhausted: 429 You exceeded your current quota, please check your plan and billing details. For more information on this error, head to: https://ai.google.dev/gemini-api/docs/rate-limits. [violations {
  quota_metric: "generativelanguage.googleapis.com/generate_content_free_tier_requests"
  quota_id: "GenerateRequestsPerDayPerProjectPerModel-FreeTier"
  quota_dimensions {
    key: "model"
    value: "gemini-2.0-flash-exp"
  }
  quota_dimensions {
    key: "location"
    value: "global"
  }
  quota_

LLM hatası: 429 You exceeded your current quota, please check your plan and billing details. For more information on this error, head to: https://ai.google.dev/gemini-api/docs/rate-limits. [violations {
  quota_metric: "generativelanguage.googleapis.com/generate_content_free_tier_requests"
  quota_id: "GenerateRequestsPerDayPerProjectPerModel-FreeTier"
  quota_dimensions {
    key: "model"
    value: "gemini-2.0-flash-exp"
  }
  quota_dimensions {
    key: "location"
    value: "global"
  }
  quota_value: 50
}
, links {
  description: "Learn more about Gemini API quotas"
  url: "https://ai.google.dev/gemini-api/docs/rate-limits"
}
, retry_delay {
  seconds: 18
}
]


127.0.0.1 - - [06/Aug/2025 15:32:40] "OPTIONS /last_response HTTP/1.1" 200 -
127.0.0.1 - - [06/Aug/2025 15:32:40] "POST /ask HTTP/1.1" 200 -
127.0.0.1 - - [06/Aug/2025 15:32:40] "GET /last_response HTTP/1.1" 200 -
127.0.0.1 - - [06/Aug/2025 15:32:41] "GET /last_response HTTP/1.1" 200 -
127.0.0.1 - - [06/Aug/2025 15:32:41] "GET /last_response HTTP/1.1" 200 -
127.0.0.1 - - [06/Aug/2025 15:32:42] "GET /last_response HTTP/1.1" 200 -
127.0.0.1 - - [06/Aug/2025 15:32:43] "GET /last_response HTTP/1.1" 200 -
127.0.0.1 - - [06/Aug/2025 15:32:44] "GET /last_response HTTP/1.1" 200 -
127.0.0.1 - - [06/Aug/2025 15:32:45] "GET /last_response HTTP/1.1" 200 -
127.0.0.1 - - [06/Aug/2025 15:32:46] "OPTIONS /last_response HTTP/1.1" 200 -
127.0.0.1 - - [06/Aug/2025 15:32:46] "GET /last_response HTTP/1.1" 200 -
127.0.0.1 - - [06/Aug/2025 15:32:47] "GET /last_response HTTP/1.1" 200 -
127.0.0.1 - - [06/Aug/2025 15:32:48] "GET /last_response HTTP/1.1" 200 -
127.0.0.1 - - [06/Aug/2025 15:32:49] "GET /last_resp

----------------------------------------------------------------------------------------------------------------------

In [ ]:
# Ana döngü
print("🚀 AI Agent ana döngüsü başlatılıyor...")

while True:
    try:
        # Veri kaynaklarından çek
        data = get_safe_json(ENDPOINT_ATTENTION)
        keyboard_data = get_safe_json(ENDPOINT_KYBRD_MOUSE_INTERRUPT)
        script_data = get_safe_json(ENDPOINT_SCRIPT)

        if not data:
            print("⏳ Veri alınamadı, bekleniyor...\n")
            time.sleep(2)
            continue

        # Oturum bilgileri
        CURRENT_USER_ID = "ogrenci_001"
        CURRENT_SESSION_ID = datetime.now().strftime("%Y%m%d_%H")
        CURRENT_TIMESTAMP = datetime.now().isoformat()

        # ——— Metadata oluştur ———
        metadata = {
            "user_id": CURRENT_USER_ID,
            "session_id": CURRENT_SESSION_ID,
            "timestamp": CURRENT_TIMESTAMP,
            "attention": data.get("attention", 0),
            "screen": data.get("head_looking_at_screen", False),
            "eye_left": data.get("left_eye_open", False),
            "eye_right": data.get("right_eye_open", False),
            "att_1min": data.get("attention_1min_avg", 0),
            "att_5min": data.get("attention_5min_avg", 0),
            "att_20min": data.get("attention_20min_avg", 0),
            "att_total": data.get("attention_total_avg", 0),
            "keyboard_activity": keyboard_data.get("keyboard_activity", False),
            "mouse_activity": keyboard_data.get("mouse_activity", False),
            "tab_changed": keyboard_data.get("tab_changed", False),
            "interaction_status": keyboard_data.get("status", "no_status"),
            # focus_score artık None da olabilir
            "focus_score": script_data.get("focus_score")
        }

        # Veriyi güvenli şekilde kaydet
        try:
            vectordb.add_texts(
                ["<sensor data>"],
                metadatas=[{**metadata, "type":"sensor"}]
            )
        except Exception as e:
            print(f"[Veri Kayıt Hatası] {e}")

        # ChromaDB temizliği (300 kayıt sınırı)
        try:
            collection_data = vectordb._collection.get()
            if len(collection_data.get("ids", [])) > 300:
                vectordb._collection.delete(ids=collection_data["ids"][:50])
                print("♻️ ChromaDB çok dolu, eski kayıtlar silindi.")
        except Exception as e:
            print(f"[ChromaDB Temizleme Hatası] {e}")

        # Karar mekanizması
        try:
            recent_docs = vectordb.similarity_search("attention", k=10)

            # Ortalama dikkat
            recent_attention = [d.metadata.get("attention", 0) for d in recent_docs]
            avg_attention = sum(recent_attention) / len(recent_attention) if recent_attention else 0

            # Ortalama focus (None değerleri atla)
            focus_vals = [
                d.metadata.get("focus_score")
                for d in recent_docs
                if isinstance(d.metadata.get("focus_score"), (int, float))
            ]
            avg_focus = sum(focus_vals) / len(focus_vals) if focus_vals else 0

            decision = None

            if metadata["attention"] < 0.3 and not metadata["screen"]:
                dikkat_uyarisi_ver(None)
                decision = "Dikkat çok düşük ve ekran dışı."

            elif metadata["att_20min"] < 0.5 and avg_attention < 0.45:
                mola_onerisi_chromadb(None)
                decision = "20dk ortalama dikkat çok düşük."

            elif avg_attention > 0.6 and metadata["attention"] < 0.4:
                zihin_yorgunlugu_tahmini(None)
                decision = "Zihin yorgunluğu olabilir."

            # focus_score None değilse ve yüksekse motivasyon ver
            elif metadata["focus_score"] is not None and metadata["focus_score"] > 0.85:
                sesli_motivasyon_ver("Odak harika! Böyle devam et.")
                decision = "Odak çok iyi, motivasyon verildi."

            # sadece gerçek bir focus_score varsa etkileşim uyarısı
            elif (not metadata["keyboard_activity"]
                  and not metadata["mouse_activity"]
                  and metadata["focus_score"] is not None):
                vak_ogrenme_tarzi_tahmini(None)
                sesli_motivasyon_ver("Biraz etkileşim gerekebilir. Hadi devam!")
                decision = "Etkileşim yoktu, öğrenme tarzı tahmini yapıldı."

            elif random.random() < 0.1:
                oturum_ozet_raporu(None)
                decision = "Rastgele özete yönlendirildi."

            if decision:
                print(f"\n🧠 Karar verildi: {decision}")

                # — Nitel tanımlamalar
                att_desc    = kvalitatif_deger(metadata["attention"])
                focus_desc  = kvalitatif_deger(metadata["focus_score"])
                interaction = "var" if (metadata["keyboard_activity"] or metadata["mouse_activity"]) else "yok"

                # — Önceki sohbetleri al
                try:
                    conv_docs = vectordb.similarity_search(
                        decision, k=5,
                        filter={"user_id": STUDENT_ID, "type": "conversation"}
                    )
                    history = "\n".join(f"{d.metadata['speaker']}: {d.page_content}" for d in conv_docs)
                except Exception as e:
                    print(f"[Sohbet Geçmişi Hatası] {e}")
                    history = ""

                # — Nitel ifadeli prompt
                prompt = f"""
        Önceki konuşmalar:
        {history}

        Durum: {decision}
        - Odak durumu: {att_desc}
        - Derin odak: {focus_desc}
        - Etkileşim: {interaction}

        Lütfen sayısal değer vermeden, sıcak ve destekleyici bir mesaj yaz.
        """
                try:
                    fb = agent.run(prompt)
                    last_agent_response = fb
                except Exception as e:
                    print(f"[Agent Hatası] {e}")
                    # Basit bir geri bildirim ver
                    last_agent_response = f"Durumun takip edildi: {decision}"
                    
        except Exception as e:
            print(f"🧠 [Zeki Koç Hatası] {e}")

    except KeyboardInterrupt:
        print("\n🛑 Program durduruldu (Ctrl+C)")
        break
    except Exception as e:
        print(f"🚨 [Ana Döngü Hatası] {e}")
        print("🔄 5 saniye bekleyip devam ediyorum...")
        time.sleep(5)
        continue

    # Döngü arası bekleme
    time.sleep(2)

🚀 AI Agent ana döngüsü başlatılıyor...

📊 Oturum Özeti:
- Ortalama dikkat: 0.66
- Ortalama focus: 0.0
- En odaklı zaman: 2025-08-06T13:21:09.584329

🧠 Karar verildi: Rastgele özete yönlendirildi.


Retrying langchain_google_genai.llms._completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised ResourceExhausted: 429 You exceeded your current quota, please check your plan and billing details. For more information on this error, head to: https://ai.google.dev/gemini-api/docs/rate-limits. [violations {
  quota_metric: "generativelanguage.googleapis.com/generate_content_free_tier_requests"
  quota_id: "GenerateRequestsPerDayPerProjectPerModel-FreeTier"
  quota_dimensions {
    key: "model"
    value: "gemini-2.0-flash-exp"
  }
  quota_dimensions {
    key: "location"
    value: "global"
  }
  quota_value: 50
}
, links {
  description: "Learn more about Gemini API quotas"
  url: "https://ai.google.dev/gemini-api/docs/rate-limits"
}
, retry_delay {
  seconds: 4
}
].


[Sohbet Geçmişi Hatası] Expected where to have exactly one operator, got {'user_id': 'ogrenci_001', 'type': 'conversation'} in query.


> Entering new AgentExecutor chain...


----------------------------------------------------------------------------------------------------------------------